In [1]:
from general_tools.notebook import gpu_utils
GPU = 2
gpu_utils.setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean
from sklearn.manifold import TSNE

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.in_out.basics import read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.nips.helper import center_pclouds_in_unit_sphere, average_per_class, zero_mean_half_sphere

from sklearn.svm import LinearSVC

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
n_pc_samples = 2048
put_in_usphere = True
model_net = '40'
top_data_dir = '/orions4-zfs/projects/optas/DATA/'


# experiment_name ='all_snc_rotated_conv_arch_2048pts_emd'
# experiment_name ='wu_classes_rotated_convolutional_arch_2048pts_chamfer'
# experiment_name ='wu_classes_rotated_convolutional_arch_2048pts_emd'
# middle_dir = 'OUT/nips/nn_models/vanilla_ae/'

# experiment_name = 'all_snc_rotated_mlp_arch_with_bnorm_on_encoder_3_pc2048pts_chamfer'
# middle_dir = 'OUT/iclr/nn_models/all_snc'

In [6]:
# train_dir = osp.join(top_data_dir, middle_dir, experiment_name)
experiment_name ='all_snc_rotated_conv_arch_2048pts_chamfer'
train_dir = '/scr/optas/DATA/nips/nn_models/vanilla_ae/all_snc_rotated_conv_arch_2048pts_chamfer/'
conf = Conf.load(osp.join(train_dir, 'configuration'))

if conf.train_dir != train_dir: # added to address moving training-folder to other location
    conf.train_dir = train_dir
    conf.save(osp.join(conf.train_dir, 'configuration'))

print conf
conf.n_output = conf.n_input
conf.allow_gpu_growth = True

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [40, 20, 10, 10], 'n_filters': [128, 128, 256, 512], 'strides': [1, 2, 2, 1]}
               experiment_name: all_snc_rotated_conv_arch_2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /scr/optas/DATA/n

In [7]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
last_epoch = saved_epochs[-1]
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

/orions4-zfs/projects/optas/Git_Repos/tf_lab/point_clouds/encoders_decoders.py:95: UserWarning: Using old architecture.
  warnings.warn('Using old architecture.')


Model restored in epoch 2000.


In [10]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [11]:
verbose = True
n_threads = 20
put_in_usphere = True
pc, model_names, labels = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = zero_mean_half_sphere(pc)
    
train_data = PointCloudDataSet(pc, labels=labels)

pc, model_names, labels = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = zero_mean_half_sphere(pc)
    
test_data = PointCloudDataSet(pc, labels=labels)

9841 pclouds were loaded. They belong in 40 shape-classes.
2467 pclouds were loaded. They belong in 40 shape-classes.


In [ ]:
train_feed, train_latent, train_classes = ae.embedding_at_tensor(train_data, conf, tensor_name='bottleneck')
cids = classes_to_integers(int(model_net), train_classes)[1]

test_feed, test_latent, test_classes = ae.embedding_at_tensor(test_data, conf, tensor_name='bottleneck')
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [ ]:
old_c_space = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
new_c_space = [2.0, 3.0, 4.0, 5.0]

c_space = new_c_space

intercept_scaling_space = [0.5, 1, 2, 3, 4]
dual_space = [False, True]
loss_space = ['hinge', 'squared_hinge']

all_scores = dict()
for c in c_space:
    for intercept in intercept_scaling_space:
        for dual in dual_space:
            for loss in loss_space:
                try:
                    lsvc = LinearSVC(C=c, loss=loss, intercept_scaling=intercept, dual=dual)
                    lsvc.fit(train_latent, cids)
                    print c, intercept, dual, loss
                    score = average_per_class(lsvc, test_latent, cids_)                
                    print score
                    all_scores[(c, intercept, dual, loss)] = score
                except:
                    continue

2.0 0.5 False squared_hinge
0.84004130967
2.0 0.5 True hinge
0.854579865361
2.0 0.5 True squared_hinge
0.839582007344
2.0 1 False squared_hinge
0.840332007344
2.0 1 True hinge
0.854620563035
2.0 1 True squared_hinge
0.841000611995
2.0 2 False squared_hinge
0.840582007344
2.0 2 True hinge
0.853554773562
2.0 2 True squared_hinge
0.840163402693
2.0 3 False squared_hinge
0.840122705018
2.0 3 True hinge
0.856289167687
2.0 3 True squared_hinge
0.836975520196
2.0 4 False squared_hinge
0.840413402693
2.0 4 True hinge
0.853723378213
2.0 4 True squared_hinge
0.839454100367
3.0 0.5 False squared_hinge
0.833103427173
3.0 0.5 True hinge
0.846744798042
3.0 0.5 True squared_hinge
0.830144124847
3.0 1 False squared_hinge
0.834394124847
3.0 1 True hinge
0.842954100367
3.0 1 True squared_hinge
0.832934822521
3.0 2 False squared_hinge
0.835434822521
3.0 2 True hinge
0.842913402693
3.0 2 True squared_hinge
0.830394124847
3.0 3 False squared_hinge
0.83501621787
3.0 3 True hinge
0.845269889841
3.0 3 True sq

In [ ]:
# Fix script to find best parameter combination
# from general_tools.simpletons import sort_dict_by_key
# sort_dict_by_key(all_scores)

# # def sort_dict_by_key(in_dict, reverse=False):
# #     return sorted(in_dict.items(), key=operator.itemgetter(0), reverse=reverse)

In [ ]:
# With WU 7 Classes:

# M10, CD, 2K epochs
# 0.4 4 True hinge
# 0.95176744186

# M40, CD, 2K epochs
# 1.0 1 True hinge
# 0.84410495716

# M10, EMD, ~2.5K epochs
# about 0.94

# M40, EMD, ~2.5K epochs
# about 0.83

In [ ]:

# USING ALL ShapeNetCore DATA : ~57K models.
# Training at (max) 2K epochs.

# EMD, 2K, model_net_40
# 1.0 3 False squared_hinge
# 0.844814259486
# 1.0 3 True squared_hinge
# 0.844814259486

# EMD, 2K, model_net_10
# 0.4 2 True hinge
# 0.953279069767


# Chamfer, 2K model_net_40
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.3 4 True squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.6 1 True squared_hinge
# 0.856264075887
# 0.7 1 False squared_hinge
# 0.856764075887

# Chamfer, 2K model_net_10
# 0.9 4 True hinge
# 0.950790697674
# 0.950790697674
# 0.8 0.5 False squared_hinge

# 1.0 4 True hinge
# 0.950790697674
# 1.0 3 True hinge
# 0.950790697674
# 0.9 4 True hinge
# 0.950790697674

# 0.7 4 True hinge
# 0.950790697674




In [ ]:
# New stuff for ICLR
# 1.  Chamfer
# encoder_args: {'filter_sizes': [1, 1, 1, 1, 1], 'strides': [1, 1, 1, 1, 1], 'n_filters': [64, 128, 128, 256, 512], 'dropout_prob': [0, 0, 0, 0, 0.5], 'b_norm': True}
# decoder_args: {'b_norm': True, 'layer_sizes': [512, 1024, 6144]}
# -> @ 980 epochs of training: best was on model-net40:
# 0.6 1 True hinge
# 0.840083537332
# -> @ 1620 epochs of training: best was on model-net40:
# 0.4 2 False squared_hinge
# 0.843783353733